Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------
# 1) Search for debt series
# ---------------------
# Example: total external debt stock in current USD (DT.DOD.DECT.CD)
series_code = "DT.DOD.DECT.CD"

# ---------------------
# 2) Fetch data for latest year
# ---------------------
data = wb.data.fetch(series_code, economy="all", time=2024)  # 2024 is example latest year
df = pd.DataFrame(data).reset_index()

# Rename and filter
df = df.rename(columns={"economy": "Country", series_code: "DebtUSD"})
df = df.dropna(subset=["DebtUSD"])

# Sort descending by debt
df = df.sort_values(by="DebtUSD", ascending=True)

# Add ranking
df["Rank"] = df["DebtUSD"].rank(method="first", ascending=False).astype(int)

# ---------------------
# 3) Plot horizontal bar chart
# ---------------------
plt.figure(figsize=(12, 12))
bars = plt.barh(df["Country"], df["DebtUSD"], color="steelblue")

plt.xlabel("Total National Debt (USD)")
plt.title("Country National Debt (Latest Year)")

for bar, rank, debt in zip(bars, df["Rank"], df["DebtUSD"]):
    width = bar.get_width()
    # Format debt label
    if debt >= 1e12:
        debt_label = f"{debt/1e12:.2f}T"
    elif debt >= 1e9:
        debt_label = f"{debt/1e9:.2f}B"
    else:
        debt_label = f"{debt:,.0f}"

    plt.text(
        width * 1.01,
        bar.get_y() + bar.get_height() / 2,
        f"#{rank} {debt_label}",
        va="center",
        fontsize=8,
    )

plt.tight_layout()
plt.show()
